In [7]:
import os
import pandas as pd

# Path to the directory containing the image files
#change the path
directory = 'C://Users/gauth/OneDrive/Desktop/DIP PROJECT/yolo_images'

# List to store extracted information
data = []

# Iterate over files in the directory
for filename in os.listdir(directory):
    # Split filename by ".chip" to get the main part
    main_part = filename.split(".chip")[0]
    
    # Split main part by "_" to extract age and gender
    age, gender, *rest = main_part.split("_")
    
    # Convert gender to string representation
    gender = "Male" if gender == "0" else "Female"
    
    # Join the rest of the filename back together (if any)
    rest = "_".join(rest)
    
    # Append extracted information to the list
    data.append({'Filename': filename, 'Age': int(age), 'Gender': gender})

# Create DataFrame from the extracted information
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


                                 Filename  Age  Gender
0    1_0_0_20161219140623097.jpg.chip.jpg    1    Male
1    1_0_0_20161219140627985.jpg.chip.jpg    1    Male
2    1_0_0_20161219140642920.jpg.chip.jpg    1    Male
3    1_0_0_20161219192208688.jpg.chip.jpg    1    Male
4    1_0_0_20161219205141196.jpg.chip.jpg    1    Male
..                                    ...  ...     ...
62  73_1_0_20170110183953439.jpg.chip.jpg   73  Female
63  73_1_1_20170120230417585.jpg.chip.jpg   73  Female
64  74_0_0_20170120225714392.jpg.chip.jpg   74    Male
65  74_0_0_20170120230055728.jpg.chip.jpg   74    Male
66  74_0_1_20170113182114430.jpg.chip.jpg   74    Male

[67 rows x 3 columns]


In [8]:
import numpy as np
import cv2
import os
import os
from ultralytics import YOLO


#loading caffe models
ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']




#path for images
folder_path = 'C://Users/gauth/OneDrive/Desktop/DIP PROJECT/yolo_images'

# List all image files in the folder
image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) ]
#No of images
print(len(image_files))


# Iterate over each image
result_list=[]
#yolo model being used for face detection
model=YOLO("yolov8n-face.pt")
for image_file in image_files:
    # Read image
    img = cv2.imread(image_file)
    #getting results yolo model
    results=model(img)
    result_list.append(results)
    boxes=results[0].boxes
    
    
    #looping over the faces detected from yolo
    for box in boxes:
        top_left_x=int(box.xyxy.tolist()[0][0])
        top_left_y=int(box.xyxy.tolist()[0][1])
        bottom_right_x=int(box.xyxy.tolist()[0][2])
        bottom_right_y=int(box.xyxy.tolist()[0][3])
        
        cv2.rectangle(img,( top_left_x,top_left_y),(bottom_right_x,  bottom_right_y),(255,0,0),2)
        detected_face=img[top_left_y:bottom_right_y,  top_left_x:bottom_right_x]
        
        #procedure for clahe equalization
        lab_img= cv2.cvtColor(detected_face, cv2.COLOR_BGR2LAB)

        #Splitting the LAB image to L, A and B channels, respectively
        l, a, b = cv2.split(lab_img)

      
        ########### basic Histogram Equlization#############
        #Apply histogram equalization to the L channel
        equ = cv2.equalizeHist(l)

      
        #Combine the Hist. equalized L-channel back with A and B channels
        updated_lab_img1 = cv2.merge((equ,a,b))

        #Convert LAB image back to color (RGB)
        hist_eq_img = cv2.cvtColor(updated_lab_img1, cv2.COLOR_LAB2BGR)

        ###########CLAHE#########################
        #Apply CLAHE to L channel
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        clahe_img = clahe.apply(l)
        
        #Combine the CLAHE enhanced L-channel back with A and B channels
        updated_lab_img2 = cv2.merge((clahe_img,a,b))

        #Convert LAB image back to color (RGB)
        CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)
        
        #Normal histogram 
        cv2.imshow('Basic equlization',hist_eq_img)
        
        cv2.imshow('CLAHE equlization', CLAHE_img)
        
        #blob input to caffe
        detected_face_blob = cv2.dnn.blobFromImage(CLAHE_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
      
        
        genderNet.setInput(detected_face_blob)
        ageNet.setInput(detected_face_blob)
        
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        
        
        agePreds=ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        
        print("gender",gender,"age",age)
        
        
        label = "{}{}".format(gender,age)
       
        cv2.putText(img, label, (top_left_x+20, top_left_y+20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
      
        #loading predicted results to the existing df
        df.loc[df['Filename'] == os.path.basename(image_file), 'Predicted_Gender'] = gender
        df.loc[df['Filename'] == os.path.basename(image_file), 'Predicted_Age'] = age
    cv2.imshow("ok",img)
  
    cv2.waitKey(0)
    

       

cv2.destroyAllWindows()    
        
        
        




67

0: 640x640 2 faces, 411.2ms
Speed: 54.3ms preprocess, 411.2ms inference, 39.8ms postprocess per image at shape (1, 3, 640, 640)
gender Male age (0-2)
gender Male age (8-12)

0: 640x640 2 faces, 201.8ms
Speed: 15.7ms preprocess, 201.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
gender Male age (25-32)
gender Male age (4-6)

0: 640x640 1 face, 182.8ms
Speed: 7.1ms preprocess, 182.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
gender Male age (25-32)

0: 640x640 1 face, 266.5ms
Speed: 8.0ms preprocess, 266.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
gender Female age (25-32)

0: 640x640 2 faces, 249.0ms
Speed: 11.7ms preprocess, 249.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
gender Male age (4-6)
gender Female age (0-2)

0: 640x640 3 faces, 287.9ms
Speed: 9.2ms preprocess, 287.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
gender Male age (0-2)
gender Male age (8-12)
g

gender Male age (25-32)

0: 640x640 (no detections), 154.8ms
Speed: 8.4ms preprocess, 154.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 faces, 136.8ms
Speed: 23.4ms preprocess, 136.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
gender Male age (4-6)
gender Male age (25-32)

0: 640x640 2 faces, 144.0ms
Speed: 15.7ms preprocess, 144.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
gender Male age (25-32)
gender Male age (4-6)

0: 640x640 2 faces, 156.3ms
Speed: 4.4ms preprocess, 156.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
gender Male age (25-32)
gender Female age (25-32)

0: 640x640 2 faces, 142.3ms
Speed: 7.4ms preprocess, 142.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
gender Female age (38-43)
gender Male age (25-32)

0: 640x640 2 faces, 158.6ms
Speed: 4.2ms preprocess, 158.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
gender Male 

In [9]:
df

,Filename,Age,Gender,Predicted_Gender,Predicted_Age
0,1_0_0_20161219140623097.jpg.chip.jpg,1,Male,Male,(8-12)
1,1_0_0_20161219140627985.jpg.chip.jpg,1,Male,Male,(4-6)
2,1_0_0_20161219140642920.jpg.chip.jpg,1,Male,Male,(25-32)
3,1_0_0_20161219192208688.jpg.chip.jpg,1,Male,Female,(25-32)
4,1_0_0_20161219205141196.jpg.chip.jpg,1,Male,Female,(0-2)
...,...,...,...,...,...
62,73_1_0_20170110183953439.jpg.chip.jpg,73,Female,Male,(25-32)
63,73_1_1_20170120230417585.jpg.chip.jpg,73,Female,NaN,NaN
64,74_0_0_20170120225714392.jpg.chip.jpg,74,Male,Male,(25-32)
65,74_0_0_20170120230055728.jpg.chip.jpg,74,Male,Female,(38-43)


In [10]:
df[40:60]

,Filename,Age,Gender,Predicted_Gender,Predicted_Age
40,31_0_0_20170117203033455.jpg.chip.jpg,31,Male,Male,(25-32)
41,31_0_0_20170120134310511.jpg.chip.jpg,31,Male,Male,(25-32)
42,31_0_0_20170120134346407.jpg.chip.jpg,31,Male,Male,(25-32)
43,32_1_0_20170110143445415.jpg.chip.jpg,32,Female,Male,(25-32)
44,32_1_0_20170111182452820.jpg.chip.jpg,32,Female,Male,(4-6)
45,32_1_1_20170103162943271.jpg.chip.jpg,32,Female,Male,(38-43)
46,32_1_2_20170103181041008.jpg.chip.jpg,32,Female,Female,(25-32)
47,32_1_2_20170103183806483.jpg.chip.jpg,32,Female,Male,(25-32)
48,32_1_2_20170104023251558.jpg.chip.jpg,32,Female,Male,(4-6)
49,32_1_2_20170104165057208.jpg.chip.jpg,32,Female,Male,(25-32)


In [11]:
# Calculate accuracy for predicted age
correct_age_predictions = 0
for index, row in df.iterrows():
    actual_age = row['Age']
    predicted_age_range = row['Predicted_Age']
    if isinstance(predicted_age_range, str):  # Check if the predicted age range is a string
        predicted_age_start, predicted_age_end = map(int, predicted_age_range.strip('()').split('-'))  # Extract start and end of predicted age range
        if actual_age >= predicted_age_start and actual_age <= predicted_age_end:
            correct_age_predictions += 1

total_images = df.shape[0]
age_accuracy = correct_age_predictions / total_images * 100



# Calculate accuracy for predicted gender
correct_gender_predictions = (df['Gender'] == df['Predicted_Gender']).sum()
gender_accuracy = correct_gender_predictions / total_images * 100

# Print results
print("Accuracy for predicted age: {:.2f}%".format(age_accuracy))
print("Accuracy for predicted gender: {:.2f}%".format(gender_accuracy))

Accuracy for predicted age: 32.84%
Accuracy for predicted gender: 53.73%
